In [1]:
import os
%pwd

'C:\\Users\\VICTUS\\Documents\\Course Material\\Krish-Naik-End-to-End-ML-Project\\Chicken-Disease-Classifier-Project\\research'

In [2]:
os.chdir('../')
%pwd

'C:\\Users\\VICTUS\\Documents\\Course Material\\Krish-Naik-End-to-End-ML-Project\\Chicken-Disease-Classifier-Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from cnn_classifier import logger
from cnn_classifier.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-23 22:26:31,932]: INFO: common: yaml file: config\config.yaml loaded successfully
[2023-09-23 22:26:31,937]: INFO: common: yaml file: params.yaml loaded successfully
[2023-09-23 22:26:31,937]: INFO: common: created directory at: artifacts
[2023-09-23 22:26:31,940]: INFO: common: created directory at: artifacts/data_ingestion
[2023-09-23 22:26:31,943]: INFO: 738703821: File already exists of size: ~ 11345 KB
